In [ ]:
%pip install datasets transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00


In [ ]:
import sklearn
import pandas as pd
import numpy as np
import random
import torch

seed = 42

np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

Getting the dataset.

In [ ]:
from datasets import load_dataset
sem_eval_2018_task_1 = load_dataset('sem_eval_2018_task_1', 'subtask5.english', trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6838 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3259 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/886 [00:00<?, ? examples/s]

Back Translation for augmentation.

In [ ]:
%pip install deep-translator --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.3 MB/s eta 0:00:00


Half of the gold standard dataset is selected to be back-translated. The augmented data, combined with the gold standard data, is saved as the augmented_data.csv

In [ ]:
from collections import Counter
from deep_translator import GoogleTranslator

# Convert to DataFrame
df = pd.DataFrame(sem_eval_2018_task_1['train'])

# Shuffle the DataFrame
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)

# Select half of the DataFrame
half_index = len(df_shuffled) // 2
half_df = df_shuffled.iloc[:half_index].copy()

# Define label columns
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
label_counts = {label: half_df[label].sum() for label in label_columns}
print("Current Label Counts:", label_counts)

# Function to perform back translation from English to French
def back_translate(text, src_lang='en', mid_lang='fr'):
    translated_text = GoogleTranslator(source=src_lang, target=mid_lang).translate(text)
    back_translated_text = GoogleTranslator(source=mid_lang, target=src_lang).translate(translated_text)
    return back_translated_text

# Augment tweets
augmented_data = []
processed_tweets = set()  # To track processed tweets

for index, row in half_df.iterrows():
    tweet = row['Tweet']
    original_labels = row[label_columns].to_dict()

    # Skip if this tweet has already been processed
    if index in processed_tweets:
        continue

    # Perform back translation
    new_tweet = back_translate(tweet)

    # Append augmented data
    augmented_data.append({'Tweet': new_tweet, **original_labels})

    # Update processed tweets set
    processed_tweets.add(index)

print(f"Number of tweets augmented: {len(augmented_data)}")

# Combine original and augmented datasets
augmented_df = pd.DataFrame(augmented_data)
combined_df = pd.concat([half_df, augmented_df], ignore_index=True)

# Verify label counts
new_label_counts = {label: combined_df[label].sum() for label in label_columns}
print("New Label Counts:", new_label_counts)

# Save combined dataset to CSV
combined_df.to_csv('augmented_data.csv', index=False)

Current Label Counts: {'anticipation': 490, 'optimism': 981, 'trust': 170, 'joy': 1245, 'love': 350, 'anger': 1241, 'disgust': 1273, 'pessimism': 401, 'sadness': 1028, 'fear': 664, 'surprise': 188}
Number of tweets augmented: 3419
New Label Counts: {'anticipation': 980, 'optimism': 1962, 'trust': 340, 'joy': 2490, 'love': 700, 'anger': 2482, 'disgust': 2546, 'pessimism': 802, 'sadness': 2056, 'fear': 1328, 'surprise': 376}


#BERT

BERT + 1/2 gold standard data.

First, we need to create a data frame that will store half of the gold standard data.

In [ ]:
# Load your dataset into a DataFrame
df = pd.DataFrame(sem_eval_2018_task_1['train'])

label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Shuffle the DataFrame
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)

# Select half of the DataFrame
half_index = len(df_shuffled) // 2
half_df = df_shuffled.iloc[:half_index].copy()

# Verify label distribution in the original and selected halves (optional)
original_label_distribution = df[label_columns].sum() / len(df)
selected_label_distribution = half_df[label_columns].sum() / len(half_df)

print("Original Label Distribution:")
print(original_label_distribution)
print("\nSelected Half Label Distribution:")
print(selected_label_distribution)

# Now `df_half` contains half of the original DataFrame with a label distribution close to the original

Original Label Distribution:
anticipation    0.143024
optimism        0.290143
trust           0.052208
joy             0.362240
love            0.102369
anger           0.372039
disgust         0.380521
pessimism       0.116262
sadness         0.293653
fear            0.181632
surprise        0.052793
dtype: float64

Selected Half Label Distribution:
anticipation    0.143317
optimism        0.286926
trust           0.049722
joy             0.364142
love            0.102369
anger           0.362972
disgust         0.372331
pessimism       0.117286
sadness         0.300673
fear            0.194209
surprise        0.054987
dtype: float64


In [ ]:
print(len(half_df))

3419


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

# Hyperparameters
num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(half_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(half_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5328, Val Loss: 0.4454
Epoch 2, Train Loss: 0.4189, Val Loss: 0.3834
Epoch 3, Train Loss: 0.3689, Val Loss: 0.3511
Epoch 4, Train Loss: 0.3320, Val Loss: 0.3435
Epoch 5, Train Loss: 0.3029, Val Loss: 0.3339
Test Accuracy: 25.28
Test F1-macro: 44.17
Test F1-micro: 66.93
Confusion Matrix:
Label: anticipation
[[2834    0]
 [ 425    0]]

Label: optimism
[[1803  313]
 [ 328  815]]

Label: trust
[[3106    0]
 [ 153    0]]

Label: joy
[[1622  195]
 [ 320 1122]]

Label: love
[[2662   81]
 [ 322  194]]

Label: anger
[[1894  264]
 [ 237  864]]

Label: disgust
[[1839  321]
 [ 261  838]]

Label: pessimism
[[2880    4]
 [ 372    3]]

Label: sadness
[[1941  358]
 [ 285  675]]

Label: fear
[[2733   41]
 [ 245  240]]

Label: surprise
[[3089    0]
 [ 170    0]]



BERT + complete gold standard data

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4745, Val Loss: 0.3885
Epoch 2, Train Loss: 0.3663, Val Loss: 0.3386
Epoch 3, Train Loss: 0.3176, Val Loss: 0.3157
Epoch 4, Train Loss: 0.2868, Val Loss: 0.3130
Epoch 5, Train Loss: 0.2645, Val Loss: 0.3055
Test Accuracy: 28.60
Test F1-macro: 48.99
Test F1-micro: 68.94
Confusion Matrix:
Label: anticipation
[[2806   28]
 [ 405   20]]

Label: optimism
[[1750  366]
 [ 235  908]]

Label: trust
[[3106    0]
 [ 153    0]]

Label: joy
[[1608  209]
 [ 247 1195]]

Label: love
[[2638  105]
 [ 272  244]]

Label: anger
[[1982  176]
 [ 296  805]]

Label: disgust
[[1930  230]
 [ 323  776]]

Label: pessimism
[[2835   49]
 [ 312   63]]

Label: sadness
[[2176  123]
 [ 433  527]]

Label: fear
[[2723   51]
 [ 180  305]]

Label: surprise
[[3089    0]
 [ 170    0]]



BERT + 1/2 gold standard data + 1/2 synthetic data

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

num_epochs = 5
batch_size = 32
learning_rate = 1e-5
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
mixed_df = pd.read_csv("augmented_data.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_columns))
model.dropout = torch.nn.Dropout(dropout)  # Adding dropout to the model
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(mixed_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(mixed_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Evaluation on test set (assuming df_test remains unchanged)
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt").to(device)
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    outputs_test = model(input_ids=inputs_test['input_ids'], attention_mask=inputs_test['attention_mask'])
    logits_test = outputs_test.logits
    predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).cpu().numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test.cpu(), predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test.cpu(), predicted_labels_test, average='micro') * 100

confusion_matrix_test = multilabel_confusion_matrix(labels_test.cpu(), predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.4872, Val Loss: 0.3842
Epoch 2, Train Loss: 0.3562, Val Loss: 0.3323
Epoch 3, Train Loss: 0.2997, Val Loss: 0.3189
Epoch 4, Train Loss: 0.2654, Val Loss: 0.3155
Epoch 5, Train Loss: 0.2388, Val Loss: 0.3194
Test Accuracy: 25.99
Test F1-macro: 48.27
Test F1-micro: 67.71
Confusion Matrix:
Label: anticipation
[[2812   22]
 [ 406   19]]

Label: optimism
[[1820  296]
 [ 341  802]]

Label: trust
[[3106    0]
 [ 153    0]]

Label: joy
[[1620  197]
 [ 266 1176]]

Label: love
[[2660   83]
 [ 296  220]]

Label: anger
[[2029  129]
 [ 354  747]]

Label: disgust
[[1957  203]
 [ 373  726]]

Label: pessimism
[[2809   75]
 [ 319   56]]

Label: sadness
[[2014  285]
 [ 315  645]]

Label: fear
[[2656  118]
 [ 130  355]]

Label: surprise
[[3089    0]
 [ 168    2]]



#RoBERTa

RoBERTa + half GSD

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4 # Adjust this based on your memory constraints and desired effective batch size

# Load dataset
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(half_df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(half_df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.6086, Val Loss: 0.4960
Epoch 2, Train Loss: 0.4854, Val Loss: 0.4731
Epoch 3, Train Loss: 0.4687, Val Loss: 0.4410
Epoch 4, Train Loss: 0.4321, Val Loss: 0.3820
Epoch 5, Train Loss: 0.4013, Val Loss: 0.3610
Epoch 6, Train Loss: 0.3788, Val Loss: 0.3477
Epoch 7, Train Loss: 0.3607, Val Loss: 0.3360
Epoch 8, Train Loss: 0.3468, Val Loss: 0.3291
Epoch 9, Train Loss: 0.3340, Val Loss: 0.3227
Epoch 10, Train Loss: 0.3245, Val Loss: 0.3263
Test Accuracy: 26.42
Test F1-macro: 45.73
Test F1-micro: 68.20
Confusion Matrix:
Label: anticipation
[[2834    0]
 [ 425    0]]

Label: optimism
[[1762  354]
 [ 276  867]]

Label: trust
[[3106    0]
 [ 153    0]]

Label: joy
[[1623  194]
 [ 301 1141]]

Label: love
[[2599  144]
 [ 237  279]]

Label: anger
[[1928  230]
 [ 254  847]]

Label: disgust
[[1870  290]
 [ 252  847]]

Label: pessimism
[[2880    4]
 [ 365   10]]

Label: sadness
[[1948  351]
 [ 277  683]]

Label: fear
[[2725   49]
 [ 251  234]]

Label: surprise
[[3089    0]
 [ 17

RoBERTa + original df

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5624, Val Loss: 0.4762
Epoch 2, Train Loss: 0.4689, Val Loss: 0.4298
Epoch 3, Train Loss: 0.4120, Val Loss: 0.3644
Epoch 4, Train Loss: 0.3734, Val Loss: 0.3447
Epoch 5, Train Loss: 0.3483, Val Loss: 0.3281
Epoch 6, Train Loss: 0.3334, Val Loss: 0.3202
Epoch 7, Train Loss: 0.3169, Val Loss: 0.3163
Epoch 8, Train Loss: 0.3083, Val Loss: 0.3130
Epoch 9, Train Loss: 0.2979, Val Loss: 0.3069
Epoch 10, Train Loss: 0.2924, Val Loss: 0.3087
Test Accuracy: 28.23
Test F1-macro: 50.47
Test F1-micro: 69.68
Confusion Matrix:
Confusion Matrix:
Label: anticipation
[[2813   21]
 [ 395   30]]

Label: optimism
[[1688  428]
 [ 195  948]]

Label: trust
[[3106    0]
 [ 153    0]]

Label: joy
[[1606  211]
 [ 226 1216]]

Label: love
[[2529  214]
 [ 182  334]]

Label: anger
[[2002  156]
 [ 327  774]]

Label: disgust
[[1962  198]
 [ 331  768]]

Label: pessimism
[[2811   73]
 [ 292   83]]

Label: sadness
[[2150  149]
 [ 397  563]]

Label: fear
[[2582  192]
 [ 116  369]]

Label: surprise


RoBERTa + half GSD + half synthetic data

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import AutoTokenizer, RobertaForSequenceClassification

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dropout = 0.2
gradient_accumulation_steps = 4

# Load dataset
df = pd.read_csv("augmented_data.csv")
df_val = pd.DataFrame(sem_eval_2018_task_1['validation'])
df_test = pd.DataFrame(sem_eval_2018_task_1['test'])
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']

# Initialize BERT tokenizer and model with dropout
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_columns), hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
model.to(device)

# Tokenize training data
inputs_train = tokenizer(list(df['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_train = torch.tensor(df[label_columns].values, dtype=torch.float32)
train_data = TensorDataset(inputs_train['input_ids'], inputs_train['attention_mask'], labels_train)
train_sampler = RandomSampler(train_data, replacement=False, num_samples=None, generator=torch.Generator().manual_seed(seed))  # Ensuring the shuffle is consistent
train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)

# Tokenize validation data
inputs_val = tokenizer(list(df_val['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_val = torch.tensor(df_val[label_columns].values, dtype=torch.float32)
val_data = TensorDataset(inputs_val['input_ids'], inputs_val['attention_mask'], labels_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for step, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Free up memory
        del input_ids, attention_mask, labels, outputs, loss
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            # Free up memory
            del input_ids, attention_mask, labels, outputs, loss
            torch.cuda.empty_cache()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

# Evaluation on test set in batches
inputs_test = tokenizer(list(df_test['Tweet']), padding=True, truncation=True, return_tensors="pt")
labels_test = torch.tensor(df_test[label_columns].values, dtype=torch.float32)

test_data = TensorDataset(inputs_test['input_ids'], inputs_test['attention_mask'], labels_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

model.eval()
all_logits = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_logits.append(logits.cpu())

        # Free up memory
        del input_ids, attention_mask, labels, outputs, logits
        torch.cuda.empty_cache()

logits_test = torch.cat(all_logits, dim=0)
predicted_labels_test = (torch.sigmoid(logits_test) > 0.5).numpy().astype(int)

# Calculate test metrics
accuracy_test = accuracy_score(labels_test, predicted_labels_test) * 100
f1_macro_test = f1_score(labels_test, predicted_labels_test, average='macro') * 100
f1_micro_test = f1_score(labels_test, predicted_labels_test, average='micro') * 100
confusion_matrix_test = multilabel_confusion_matrix(labels_test, predicted_labels_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test F1-macro: {f1_macro_test:.2f}")
print(f"Test F1-micro: {f1_micro_test:.2f}")
print("Confusion Matrix:")
for label, matrix in zip(label_columns, confusion_matrix_test):
    print(f"Label: {label}")
    print(matrix)
    print()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train Loss: 0.5539, Val Loss: 0.4733
Epoch 2, Train Loss: 0.4500, Val Loss: 0.3866
Epoch 3, Train Loss: 0.3903, Val Loss: 0.3520
Epoch 4, Train Loss: 0.3607, Val Loss: 0.3354
Epoch 5, Train Loss: 0.3373, Val Loss: 0.3246
Epoch 6, Train Loss: 0.3180, Val Loss: 0.3161
Epoch 7, Train Loss: 0.3046, Val Loss: 0.3142
Epoch 8, Train Loss: 0.2929, Val Loss: 0.3090
Epoch 9, Train Loss: 0.2842, Val Loss: 0.3079
Epoch 10, Train Loss: 0.2731, Val Loss: 0.3059
Test Accuracy: 28.63
Test F1-macro: 49.27
Test F1-micro: 69.65
Confusion Matrix:
Label: anticipation
[[2809   25]
 [ 396   29]]

Label: optimism
[[1777  339]
 [ 256  887]]

Label: trust
[[3106    0]
 [ 153    0]]

Label: joy
[[1615  202]
 [ 246 1196]]

Label: love
[[2591  152]
 [ 224  292]]

Label: anger
[[1916  242]
 [ 248  853]]

Label: disgust
[[1881  279]
 [ 260  839]]

Label: pessimism
[[2852   32]
 [ 333   42]]

Label: sadness
[[2150  149]
 [ 401  559]]

Label: fear
[[2684   90]
 [ 170  315]]

Label: surprise
[[3088    1]
 [ 17